In [1]:
import pandas as pd
import ast

In [6]:
df_ZS=pd.read_csv('data/restaurants_zlatestranky.csv')
df_API=pd.read_csv('data/restaurants_Places_API.csv')

In [7]:
#adding empty column to df_ZS
empty_list = []
for i in df_ZS.index:
    empty_list.append(None)
df_ZS["exact_match"] = empty_list

In [47]:
#comparison loop
for i in df_ZS.index:
    if df_ZS["name"][i] == "Céleste":  #assigns None in case of this outiler
        celeste_exception = None
        df_ZS.exact_match.iat[i] = celeste_exception

    elif pd.isna(df_API["name"][i]):   #assigns None is case of no resaurant found by Google API
        df_ZS.exact_match.iat[i] = None
        
    else:                              #assigns true in case of match and False in case of no match
        zs_phones_dict = ast.literal_eval(df_ZS["phones"][i])
        
        zs_phones_list = []
        for n in zs_phones_dict:
            net_number = zs_phones_dict[n][5:]
            zs_phones_list.append(net_number)

        if df_API["formatted_phone_number"][i] in zs_phones_list:
            df_ZS.exact_match.iat[i] = True
        else:
            df_ZS.exact_match.iat[i] = False
            


In [49]:
df_ZS.exact_match.describe()

count     1101
unique       2
top       True
freq       799
Name: exact_match, dtype: object

In [51]:
df_ZS.describe()

,Unnamed: 0,ratings,review_count
count,2192.000000,2192.000000,2192.000000
mean,1095.500000,6.801551,0.128650
std,632.920216,23.540934,0.535257
min,0.000000,0.000000,0.000000
25%,547.750000,0.000000,0.000000
50%,1095.500000,0.000000,0.000000
75%,1643.250000,0.000000,0.000000
max,2191.000000,100.000000,8.000000
